<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [73]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [6]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [35]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)


In [139]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, ngrams=2, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(cleaned_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=50, output_dim=10)

    def clean_text(self, text):
        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return embedded_text

# Example usage with a batch of texts
batch_texts = tf.constant([text])
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)

for word in cleaned_batch[0]:
  print(word)



tf.Tensor(
[ 0.16625446 -0.10690974 -0.0959322   0.05431825 -0.13190755 -0.02251297
  0.22102675 -0.18265738  0.03327501  0.27576146], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.08394521 -0.05697772  0.18987873 -0.08254093  0.02117789 -0.04704982
 -0.07318649  0.3102691  -0.19164124  0.28716126], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.02755988 -0.06913637  0.0655427   0.30821308  0.18436924  0.29598287
 -0.26427546 -0.26950228  0.00336063  0.19067577], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.24231151  0.10123417  0.05342883 -0.26772553  0.14305812  0.10481837
  0.30369923  0.08833742  0.25195614  0.18055424], shape=(10,), dtype=float32)
tf.Tensor(
[-0.20690095 -0.02963632  0.1782839   0.07781541 -0.07699859  0.1231415
  0.27043083 -0.28704834  0.2754037   0.10570401], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.31579372 -0.21370637 -0.2247988  -0.16260906  0.2164245   0.22542581
 -0.18624201  0.09324574  0.07257167 -0.00042477], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.20652083